<a href="https://colab.research.google.com/github/elthonf/plataformas-cognitivas-docker/blob/master/Concess%C3%A3o_de_Cr%C3%A9dito_com_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configurações

![alt text](https://github.com/elthonf/plataformas-cognitivas-docker/blob/master/images/capa.png?raw=1)

In [0]:
#@title Parâmetros da solução
#@markdown Informe a quantidade de clientes que devem ser analisados.
qtde_clientes = 4  #@param {type: "slider", min: 1, max: 10}

#@markdown Informe o IP do servidor de predição da sua solução:
my_server_ip = "104.197.138.163" #@param {type:"string"}

#@markdown Informe a porta do servidor de predição da sua solução, se nada foi alterado, deve ser 443:
my_server_port = "443" #@param {type:"string"}


#@markdown Ah, e não se esqueça de executar esta célula!


In [0]:
import requests
import pandas as pd
import json

# Obtenção dos empréstimos pendentes de análise

Vamos agora obter os clientes que estã precisand de empréstimo e exibí-los na tela

In [0]:
url = "https://us-central1-emf-teacher.cloudfunctions.net/function-aulas-getclient?qtde={0}".format(qtde_clientes)
headers = {'Content-Type': 'application/json'}
response = requests.request("GET", url, headers=headers)
response_content = response.text.encode('utf8').decode()
clientes = pd.read_json(response_content)

In [0]:
clientes

,nome,renda,idade,etnia,sexo,casapropria,outrasrendas,estadocivil,escolaridade
22672,"Daniel, Wu",4028.099732,35.959014,1,0,1,1,0,2
3029,"Brian, Wright",3290.165416,37.610771,1,0,0,0,0,2
19240,"Andre, Dave",2919.327934,20.430507,1,0,1,0,0,1
44198,"Alma Ruby, Cordova",2741.916209,28.612836,1,1,0,0,1,1


# Predição

O cliente agora será submetido a dois modelos de Machine Learning preparados para a análise.

In [0]:
url = "http://{}:{}/predict".format(my_server_ip, my_server_port)
headers = {'Content-Type': 'application/json'}
conteudo = clientes.to_json()

In [0]:
response01 = requests.request("POST", url + "?model=modelo01", headers=headers, data=conteudo)
respostas01 = json.loads(response01.text.encode('utf8').decode())
respostas01

{'prediction': [0, 1, 1, 1],
 'proba': [[0.9712613311116339, 0.02873866888836604],
  [0.10609475169972526, 0.8939052483002747],
  [0.16488817236745124, 0.8351118276325488],
  [0.006931977891545316, 0.9930680221084547]]}

In [0]:
df_01 = pd.Series(respostas01['prediction'], index = clientes.index, name="PredicaoML01")
df_01B = pd.Series(respostas01['proba'], index = clientes.index, name="PredicaoML01B")

In [0]:
response02 = requests.request("POST", url + "?model=modelo02", headers=headers, data=conteudo)
respostas02 = json.loads(response02.text.encode('utf8').decode())
respostas02

{'prediction': [-0.010159329576254223,
  0.5917030901875656,
  0.6236547836165044,
  1.0260868020451164]}

In [0]:
df_02 = pd.Series(respostas02['prediction'], index = clientes.index, name="PredicaoML02")

In [0]:
pd.DataFrame(clientes.loc[:,"nome"])\
    .merge(right= df_01, left_index=True, right_index=True)\
    .merge(df_01B, left_index=True, right_index=True)\
    .merge(df_02, left_index=True, right_index=True)

,nome,PredicaoML01,PredicaoML01B,PredicaoML02
22672,"Daniel, Wu",0,"[0.9712613311116339, 0.02873866888836604]",-0.010159
3029,"Brian, Wright",1,"[0.10609475169972526, 0.8939052483002747]",0.591703
19240,"Andre, Dave",1,"[0.16488817236745124, 0.8351118276325488]",0.623655
44198,"Alma Ruby, Cordova",1,"[0.006931977891545316, 0.9930680221084547]",1.026087
